# Introdução à recuperação de informações

## Lista de Exercícios 3 - Recuperação probabilística

In [263]:
import nltk
import os
import re
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import gutenberg, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.probability import FreqDist
from nltk.stem.snowball import EnglishStemmer
import string
from string import punctuation
from pprint import pprint
from collections import Counter
from collections import defaultdict
import numpy as np
import math

Vamos partir da Prática 3 de modelagem de assuntos. Vamos usar a técnica de LSI para definir um conjunto de documentos relevantes.

### Exercício 1:
A partir de um corpus à sua escolha, estime um modelo de assuntos baseado no Modelo LSI. Uma vez calculado o modelo, defina um conjunto de **documentos relevantes (${\cal R}$)** para um assunto, como os $n$ documentos que contiverem em sua representação LSI, os maiores coeficientes para o assunto escolhido. Construa uma consulta $q$, com as dez palavras mais importantes do assunto escolhido.

Neste exercício, vou utilizar o corpora Gutenberg, que contem obras literárias do Projeto Gutenberg.

In [175]:
nltk.download('stopwords')
nltk.download('gutenberg')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoniohof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\antoniohof\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [176]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [177]:
textos = [gutenberg.raw(id) for id in gutenberg.fileids()]
tok_textos = [WordPunctTokenizer().tokenize(t.lower()) for t in textos]

T = nltk.TextCollection(tok_textos)

In [178]:
swu = stopwords.words('english')+ list(punctuation)

In [179]:
dicionario = corpora.Dictionary(tok_textos)
dicionario.save('dic_gutenberg.dict')
print(dicionario)

Dictionary(42339 unique tokens: ['guided', 'unopen', 'whereover', 'pembroke', 'jehonathan']...)


In [180]:
corpus = [dicionario.doc2bow(d, allow_update=True) for d in tok_textos]

In [181]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [182]:
lsi = models.LsiModel(corpus_tfidf, id2word=dicionario, num_topics=10)
corpus_lsi = lsi[corpus_tfidf]

In [183]:
lsi.show_topics(10,10)

[(0,
  '0.565*""" + 0.185*"."" + 0.179*","" + 0.143*"flambeau" + 0.141*"syme" + 0.137*"?"" + 0.125*"have" + 0.122*"--" + 0.121*"mr" + 0.116*"turnbull"'),
 (1,
  '0.425*"haue" + 0.256*"macb" + 0.197*"ham" + 0.190*"bru" + 0.175*"vs" + 0.158*"vpon" + 0.153*"brutus" + 0.136*"selfe" + 0.133*"thee" + 0.133*"cassi"'),
 (2,
  '-0.288*"}" + -0.262*"thee" + -0.195*"unto" + -0.183*"thou" + 0.169*"haue" + 0.160*""" + -0.159*"thel" + -0.135*"thy" + -0.123*"heaven" + -0.112*"weep"'),
 (3,
  '-0.309*"elinor" + 0.277*"syme" + 0.260*"buster" + -0.237*"elliot" + -0.228*"emma" + 0.213*"turnbull" + -0.204*"mrs" + -0.198*"marianne" + -0.179*"wentworth" + -0.175*"harriet"'),
 (4,
  '-0.759*"alice" + -0.235*"buster" + -0.208*",\'" + -0.205*"!\'" + 0.153*"syme" + 0.130*"turnbull" + -0.129*"duchess" + -0.128*"gryphon" + -0.123*"dormouse" + -0.114*"?\'"'),
 (5,
  '0.687*"buster" + -0.291*"alice" + 0.231*"joe" + 0.205*"blacky" + -0.198*"syme" + 0.154*"billy" + -0.139*"turnbull" + 0.126*"sammy" + 0.125*"otter" + 

Dentre os 10 tópicos gerados, vou escolher o assunto 9 para então definir os documentos relevantes:

In [184]:
assunto = lsi.print_topic(9)
assunto = re.sub("\d+", "", assunto)
assunto = re.sub('[^0-9a-zA-Z ]', '', assunto)
print(assunto)

elinor  marianne  elliot  wentworth  dashwood  jennings  anne  syme  turnbull  musgrove


Vamos fazer a busca por similaridade do assunto 9, escolhido entre os 20 listados acima

In [185]:
vec_bow = dicionario.doc2bow(assunto.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 0.74073998283605225), (1, -0.20728452005182377), (2, 0.46205101744232624), (3, -0.89284993885431163), (4, 0.19356452767943028), (5, -0.043317220736093121), (6, 0.40978418890312363), (7, -0.13347134476203454), (8, -0.29965867643897148), (9, -0.37081386716054776)]


In [186]:
index = similarities.MatrixSimilarity(corpus_lsi) 

In [187]:
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[(2, 0.80243045), (11, 0.67341632), (0, 0.64102715), (9, 0.45198274), (1, 0.41647601), (5, 0.36444372), (10, 0.20737492), (3, 0.11719015), (14, 0.0022857334), (16, 0.0014392686), (15, 0.00022700906), (8, -0.0036905892), (4, -0.028835736), (7, -0.056651626), (13, -0.064940497), (12, -0.065045677), (17, -0.14795336), (6, -0.19265674)]


#### Lista de documentos relevantes (R):
* defini como critério coeficiente LSI >= 0,3 (o corpora não é muito grande, se eu restringir muito teria poucos documentos relevantes)

In [188]:
R = []
for i in sims:
        if i[1] >= 0.3:
            R.append(i[0])        

In [174]:
print(R)

[2, 11, 0, 9, 1, 5]


#### Definindo a consulta com as 10 palavras mais importantes do assunto:

In [148]:
q = ['elinor', 'marianne', 'elliot', 'wentworth', 'dashwood', 'jennings', 'anne', 'syme', 'turnbull', 'musgrove']

### Exercício 2: 
Reutilizando os índices invertidos construídos em exercícios anteriores(Booleano, e TFIDF), calcule a precisão e revocação  com a consulta $q$ e o conjunto relevante ${\cal R}$ definidos no exercício anterior.

#### Consulta booleana:

In [118]:
booleano = defaultdict(lambda:set([]))
for tid,t in enumerate(tok_textos):
    for term in t:
        booleano[term].add(tid)

In [149]:
resultado_b = []
for k in q:
    numero = booleano[(k)]
    for j in numero:
        if j not in resultado_b:
            resultado_b.append(j)

print(resultado_b)

[2, 11, 1, 0, 8, 10, 12]


Para calcular a Revocação e a Precisão, precisamos encontrar a interseção entre os documentos relevantes e os documentos encontrados:

In [123]:
def intersecao(R, resultado_b):
    return list(set(R) & set(resultado_b))

print(intersecao(R, resultado_b))

[0, 1, 2, 11]


In [121]:
precisao = len(intersecao(R, resultado_b))/len(resultado_b)
revocacao = len(intersecao(R, resultado_b))/len(R)

print('Precisão = ', round(precisao, 3), 'Revocação = ', round(revocacao,3))


Precisão =  0.571 Revocação =  0.667


#### Consulta por TF-IDF:

In [151]:
tfidf_matrix = np.zeros((len(tok_textos),len(q)))
for j,w in enumerate(consulta):
    for i, d in enumerate(tok_textos):
        tfidf_matrix[i,j] = T.tf_idf(w,d)
        
from numpy.linalg import norm
MN = np.array([r/norm(r) if norm(r) !=0 else np.zeros(len(r)) for r in tfidf_matrix])
def ordem(q):
    return [np.dot(q,r) for r in MN]
qv = np.array([T.tf_idf(w,consulta) for w in q])
qv /= norm(qv)
r = ordem(qv)
v=filter(lambda x : x[0]!=0.0, zip(r,range(len(tok_textos))))

tfidf = sorted(v, reverse=True) 
print(tfidf)

[(0.57772935998400421, 2), (0.56084409828848547, 1), (0.33946379482129985, 8), (0.33940082240258135, 10), (0.27872767279947253, 11), (0.11088026188415426, 12), (0.11088026188415426, 0)]


In [140]:
resultado_tfidf = []
for i in tfidf:
    resultado_tfidf.append(i[1])
    
print(resultado_tfidf)

[2, 1, 8, 10, 11, 12, 0]


Para calcular a Revocação e a Precisão, precisamos encontrar a interseção entre os documentos relevantes e os documentos encontrados:

In [141]:
def intersecao(R, resultado_tfidf):
    return list(set(R) & set(resultado_tfidf))

print(intersecao(R, resultado_tfidf))

[0, 1, 2, 11]


In [143]:
precisao = len(intersecao(R, resultado_tfidf))/len(resultado_tfidf)
revocacao = len(intersecao(R, resultado_tfidf))/len(R)

print('Precisão = ', round(precisao, 3), 'Revocação = ', round(revocacao,3))

Precisão =  0.571 Revocação =  0.667


### Exercício 3: 
Usando as definições de probabilidade de relevância apresentadas no capítulo 11 do Livro, construa uma função de recuperação probabilística usando o *log da razão de Odds* como **RSV** (retrieval status value). Calcule revocação e precisão para consulta $q$ e conjunto relevante ${\cal R}$. Compare a probabilidade $p_t=P(x_t=1|R=1,q)$, com a o rankeamento de importância das palavras que compõem o assunto escolhido.

O *log da razão de Odds* para os termos da consulta $c_t$ é dado por:

\begin{equation}
\nonumber
c_t = \log \frac{p_t(1-u_t)}{u_t(1-p_t)} = \log \frac{p_t}{1-p_t} - \log \frac{u_t}{1-u_t}
\end{equation}


Em que:

i) $x_1 = p_t/(1-p_t)$ são os odds do termo aparecer se o documento for relevante; e

ii) $x_2 = u_t/(1-u_t)$ são os odds do termo aparecer se o documento for irrelevante 



RSV para o documento d: $RSV = ∑ c_t$ 







Vamos utilizar a mesma consulta definida no Exercício 1:

In [152]:
q = ['elinor', 'marianne', 'elliot', 'wentworth', 'dashwood', 'jennings', 'anne', 'syme', 'turnbull', 'musgrove']

In [ ]:
booleano = defaultdict(lambda:set([])) 
for tid,t in enumerate(tok_textos):
    for term in t:
        booleano[term].add(tid)

In [260]:
termo1 = []
termo2 = []
resultado = []
for k in q:
    numero = booleano[(k)]
    for j in numero:
        if j not in resultado:
            resultado.append(j)
    
    def intersecao(R, resultado):  #p_t: termo presente em doc relevante
        return list(set(R) & set(resultado))
    
    x1 = np.log((len(intersecao(R, resultado))/len(R))/(1-(len(intersecao(R, resultado))/len(R))))
    
    termo1.append(x1)
    
    not_intersection = list(set(resultado).difference(R))  #u_t: termo presente em doc irrelevante

    not_R = []
    for i in sims:  #definindo o conjunto de docs não relevantes
        if i[1] < 0.3:
            not_R.append(i[0])   
    if len(not_intersection) == 0:
        x2 = 0
    else:
        x2 = np.log((len(not_intersection)/len(not_R)) / (1-(len(not_intersection)/len(not_R))))
    termo2.append(x2)

print('Lista com o termo 1 para cada palavra da consulta:', termo1 , '                                                    '
      'Lista com o termo 2 para cada palavra da consulta:', termo2,)

Lista com o termo 1 para cada palavra da consulta: [-1.6094379124341005, -0.6931471805599454, 0.0, 0.0, 0.0, 0.0, 0.69314718055994518, 0.69314718055994518, 0.69314718055994518, 0.69314718055994518]                                                     Lista com o termo 2 para cada palavra da consulta: [0, 0, 0, 0, 0, 0, -1.0986122886681098, -1.0986122886681098, -1.0986122886681098, -1.0986122886681098]


In [261]:
soma_ct = []
for k in range(10):
    ct = termo1[k] - termo2[k]
    soma_ct.append(ct)
    
print(soma_ct)

[-1.6094379124341005, -0.6931471805599454, 0.0, 0.0, 0.0, 0.0, 1.791759469228055, 1.791759469228055, 1.791759469228055, 1.791759469228055]


In [262]:
RSV = 0
for k in range(10):
    RSV = RSV+soma_ct[k]
    
print(round(RSV,3))

4.864


Professor, tentei fazer seguindo os slides, mas acho que não fiz certo, então não soube continuar.

### Exercício 4:
Repita o exercício 3 agora usando o modelo **Okapi BM25** para o rankeamento. Calcule precisão e revocação.

Professor, tentei aprender a utilizar o modelo seguindo os slides e 
algumas implementaçòes sugeridas na internet mas não consegui.

In [282]:
self = textos

In [283]:
class BM25 :
    def __init__(self, fn_docs, delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        raw_data = []
        for line in file(self.fn_docs) :
            raw_data.append(line.strip().split(self.delimiter))
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        for line in file(self.fn_docs) :
            doc = line.strip().split(self.delimiter)
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items
